# Analysis of Reviews on Olist

🎯 Now that you are familiar with NLP, let's analyze the reviews of Olist.

👇 Run the following cell to load the reviews dataset and install `unidecode`

In [1]:
!pip install -q unidecode

import pandas as pd

url = "https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ml_olist_nlp_reviews.csv"
df = pd.read_csv(url, low_memory = False)

df.head()


,Unnamed: 0,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


In [2]:
df.shape

(98657, 17)

❓ **Question: Analyse the reviews to understand what could be the causes of the bad review scores** ❓

This challenge is not as guided as the previous ones. But here are some questions to ask yourself:

- Are all the reviews relevant ? 
- What about combining the title and the body of a review ?
- What cleaning operations would you apply to the reviews ?

In [7]:
data=df.copy()
data.dropna(subset=['review_comment_message'], inplace=True)

In [12]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('portuguese'))

def preprocessing(sentence):
    sentence = sentence.strip()
    sentence = sentence.lower()
    sentence =''.join(char for char in sentence if not char.isdigit())
    for punctuation in string.punctuation:
        sentence=sentence.replace(punctuation, '')
    words = word_tokenize(sentence)
    words=[word for word in words if not word in stop_words]
    words = [
        WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
        for word in words
    ]
    words = [
        WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
        for word in words
    ]
    return ' '.join(words)

In [14]:
data['clean_reviews']=data['review_comment_message'].apply(preprocessing)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=2)

# Training it on the texts
weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(data['clean_reviews']).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
weighted_words

,aa,aaa,ab,abaixar,abaixar frete,abaixo,abaixo esperado,abaixo expectativa,abaixo qualidade,abaixo valor,...,úteis entrega,úteis entregue,úteis porém,úteis prazo,úteis produto,úteis recomendo,úteis super,útil,útil após,útil recomendo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer_n_gram = CountVectorizer(ngram_range = (1,2), min_df=3) # BI-GRAMS
clean_reviews_n_gram = count_vectorizer_n_gram.fit_transform(data['review_comment_message'])

# Show the representations in a nice DataFrame
clean_reviews_n_gram = pd.DataFrame(
    clean_reviews_n_gram.toarray(),
    columns = count_vectorizer_n_gram.get_feature_names_out(),
    index = data['review_comment_message']
)

# Show the vectorized movies with bigrams
clean_reviews_n_gram

,00,00 cada,00 de,00 mais,00 nâo,00 reais,000,000 btus,001689,001689 aguardo,...,único porém,único problema,único produto,único que,único senão,úteis,úteis para,úteis que,útil,útil na
review_comment_message,,,,,,,,,,,,,,,,,,,,,
Recebi bem antes do prazo estipulado.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Mas um pouco ,travando...pelo valor ta Boa.\r\n",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Vendedor confiável, produto ok e entrega antes do prazo.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
para este produto recebi de acordo com a compra realizada!,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Entregou dentro do prazo. O produto chegou em condições perfeitas. Estou muito satisfeita!,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"O produto não foi enviado com NF, não existe venda sem NF, com certeza fico no aguardo do envio da NF podendo ser por e-mail.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
key_words=['producto errado','ainda nao','nao entregue','nao veio','nao gostei','produto defeito','nao functiona','produto diferente','pessima qualidade','veio defeito','veio faltando','veio errado']
for k in key_words:
    if k in clean_reviews_n_gram:
        print(k+': '+str(clean_reviews_n_gram[k].sum()))

ainda nao: 138
nao entregue: 11
nao veio: 45
nao gostei: 41
produto diferente: 60
pessima qualidade: 16
veio faltando: 130
veio errado: 107


🇧🇷 Some Brazilian expressions and their translations:

- `producto errado` = wrong product
- `ainda nao` = not yet
- `nao entregue` = not delivered
- `nao veio` = did not come
- `nao gostei` = did not like it
- `produto defeito` = defective product
- `nao functiona` = not working
- `produto diferente` = different product
- `pessima qualidade` = poor quality
- `veio defeito` = came defect
- `veio faltando` = came missing
- `veio errado` = came wrong

🏁 Congratulations. Instead of reading 90K+ reviews, you were able to detect the main reasons of dissatisfactions on Olist.

💾 Don't forget to `git add/commit/push`